Setup

In [3]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from tensorflow_docs.vis import embed

from tensorflow import keras
from keras import layers

from abc import ABC, abstractmethod

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import sys
import os

Data preparation

In [32]:
TEST_RATIO = 0.3
VALIDATION_RATIO = 0.25
TRAIN_RATIO = 1 - TEST_RATIO - VALIDATION_RATIO

HOLDOUT_RATIO = 0.04

MAX_SEQ_LENGTH = 150
IMG_SIZE = 128

BATCH_SIZE = 128
CLASSES = 4

EPOCHS = 40#40

#FEATURE_DIR = f""
#FEATURE_DIR = f"../Features/InceptionV3_SEQUENCE/{MAX_SEQ_LENGTH}"
MODEL_DIR = f"../Models/List"


SYS_OUT = sys.stdout

In [3]:
class CNN_Model(ABC):

    @abstractmethod
    def Get_model_name():
        pass
    @abstractmethod
    def Get_feature_num():
        pass

    @abstractmethod
    def Get_features_dir_seq():
        pass
    @abstractmethod
    def Get_features_dir_ind():
        pass
    @abstractmethod
    def Get_features_dir_frame():
        pass

class InceptionV3(CNN_Model):  
    def Get_model_name():
        return 'InceptionV3'  
    def Get_feature_num():
        return 2048
    @abstractmethod
    def Get_features_dir_seq():
        return f'../Features/InceptionV3/SEQUENCE/{MAX_SEQ_LENGTH}'
    def Get_features_dir_ind():
        return f'../Features/InceptionV3/NORMAL/'
    def Get_features_dir_frame():
        return f'../Features/InceptionV3/FRAME/'
    
class ResNet101V2(CNN_Model): 
    def Get_model_name():
        return 'ResNet101V2'
    def Get_feature_num():
        return 2048
        #return 1024
    @abstractmethod
    def Get_features_dir_seq():
        return f'../Features/ResNet101V2/SEQUENCE/{MAX_SEQ_LENGTH}'
    def Get_features_dir_ind():
        return f'../Features/ResNet101V2/NORMAL/'
    def Get_features_dir_frame():
        return f'../Features/ResNet101V2/FRAME/'
    
class DenseNet121(CNN_Model): 
    def Get_model_name():
        return 'DenseNet121'
    def Get_feature_num():
        return 1024
    @abstractmethod
    def Get_features_dir_seq():
        return f'../Features/DenseNet121/SEQUENCE/{MAX_SEQ_LENGTH}'
    def Get_features_dir_ind():
        return f'../Features/DenseNet121/NORMAL/'
    def Get_features_dir_frame():
        return f'../Features/DenseNet121/FRAME/'
    
class MobileNetV2(CNN_Model):
    def Get_model_name():
        return 'MobileNetV2'
    def Get_feature_num():
        return 1280
    @abstractmethod
    def Get_features_dir_seq():
        return f'../Features/MobileNetV2/SEQUENCE/{MAX_SEQ_LENGTH}'
    def Get_features_dir_ind():
        return f'../Features/MobileNetV2/NORMAL/'
    def Get_features_dir_frame():
        return f'../Features/MobileNetV2/FRAME/'
    
class MobileNetV3(CNN_Model):
    def Get_model_name():
        return 'MobileNetV3'
    def Get_feature_num():
        return 576
    @abstractmethod
    def Get_features_dir_seq():
        return f'../Features/MobileNetV3/SEQUENCE/{MAX_SEQ_LENGTH}'
    def Get_features_dir_ind():
        return f'../Features/MobileNetV3/NORMAL/'
    def Get_features_dir_frame():
        return f'../Features/MobileNetV3/FRAME/'
    
class MobileNetV3_Large(CNN_Model):
    def Get_model_name():
        return 'MobileNetV3_Large'
    def Get_feature_num():
        return 960
    @abstractmethod
    def Get_features_dir_seq():
        return f'../Features/MobileNetV3_Large/SEQUENCE/{MAX_SEQ_LENGTH}'
    def Get_features_dir_ind():
        return f'../Features/MobileNetV3_Large/NORMAL/'
    def Get_features_dir_frame():
        return f'../Features/MobileNetV3_Large/FRAME/'

In [7]:
#CNN = DenseNet121
#CNN = ResNet101V2
CNN = MobileNetV3_Large

Load Data

In [8]:
def Get_directory_size(url):
    list_dir = os.listdir(url)
    size = sum(os.path.isdir(os.path.join(url, x)) for x in list_dir)
    return size

def Get_file_amount(url):
    list_dir = os.listdir(url)
    return len(list_dir)

def LoadData(data, d, m, seq, stride, num_features):

    size = data.shape[0]
    i = stride
    lim = 0

    while (lim < size):
        lim = i + seq
        if (lim < size):
            d = np.append(d, [data[i:lim]], axis=0)
            m = np.append(m, [np.ones((seq))], axis=0)
        else:
            if ((size-i) / (lim-i) > 0.2):
                fix = np.zeros((lim - size, num_features))
                data = np.vstack((data, fix))

                m_ones = np.ones((size - i))
                m_ones = np.append(m_ones, np.zeros((lim - size)))

                d = np.append(d, [data[i:lim]], axis=0)
                m = np.vstack((m, m_ones))

        i = i + seq
    return d, m

def LoadVideo(data_dir, labels_dir, seq, stride, num_features):
    data = np.load(data_dir)
    labels = np.load(labels_dir)
    
    d = np.zeros((0, seq, num_features))
    m = np.zeros((0, seq))

    i = 0
    long = int(data.shape[0]/stride)
    while (i <= long):
        d, m = LoadData(data, d, m, seq, i*stride, num_features)
        i = i + 1
    l = np.ones((d.shape[0], 1)) * labels[0]
    return d, m, l

def LoadVideo_SIMPLE(dir):
    label_value = np.load(f'{dir}/label.npy')
    d = np.load(f'{dir}/data.npy')
    m = np.ones((d.shape[0]))
    l = np.ones((d.shape[0], 1)) * label_value

    return d, m, l

def LoadVideo_SIMPLE_(data_dir, labels_dir):
    label_value = np.load(labels_dir)
    d = np.load(data_dir)
    m = np.ones((d.shape[0]))
    l = np.ones((d.shape[0], 1)) * label_value

    return d, m, l

Cargar Dataset dividiendo los datos primero

In [28]:
def DatasetCount(dir, dataset_list):
    label_dir = f'{dir}\label.npy'

    if (os.path.exists(label_dir)):
        label_idx = int(np.load(label_dir)[0])
        dataset_list[label_idx].append(dir)
    else:
        list = os.listdir(dir)

        for l in list:
            url = os.path.join(dir, l)
            if os.path.isdir(url):
                DatasetCount(url, dataset_list)

def DatasetCopyRegister(dataset_list, p0, p1):    
    dataset_ready = []
    for i in range(CLASSES):
        init = int(len(dataset_list[i]) * p0)
        size = int(len(dataset_list[i]) * p1)
        for j in range(init, size):
            dataset_ready.append(dataset_list[i][j])
        print(f'CLASS: {i}, {init}_{size}')
    print()
    return dataset_ready

def LoadDatasetRandom(dir, cnn, LoadData, seq, stride):
    dataset_list = [[],[],[],[]]

    DatasetCount(dir, dataset_list)

    for i in range(len(dataset_list)):
        random.shuffle(dataset_list[i])

    dataset_ready = [[],[],[]]

    dataset_ready[2] = DatasetCopyRegister(dataset_list, TRAIN_RATIO + TEST_RATIO, 1)
    dataset_ready[1] = DatasetCopyRegister(dataset_list, TRAIN_RATIO, TRAIN_RATIO + TEST_RATIO)
    dataset_ready[0] = DatasetCopyRegister(dataset_list, 0, TRAIN_RATIO)

    train_data, train_labels = LoadData(dataset_ready[0], cnn, seq, stride, False)
    test_data, test_labels = LoadData(dataset_ready[1], cnn, seq, stride, False)
    val_data, val_labels = LoadData(dataset_ready[2], cnn, seq, stride, False)

    return train_data, train_labels, test_data, test_labels, val_data, val_labels

Frames Individuales

In [10]:
def LoadDatasetDirection_Frame(dir, cnn, seq, stride, augmented):
    size = len(dir)    
    data = np.zeros((0, cnn.Get_feature_num()))
    mask = np.zeros((0))
    labels = np.zeros((0, 1))

    for idx in range(size):
        d, m, l = LoadVideo_SIMPLE_(f'{dir[idx]}/data.npy', f'{dir[idx]}/label.npy')
        data = np.vstack((data, d))
        mask = np.append(mask, m)
        labels = np.vstack((labels, l))
        if (augmented):
            i = 0
            while (os.path.exists(f'{dir[idx]}/data_m{i}.npy') and i <= 2):          
                d, m, l = LoadVideo_SIMPLE_(f'{dir[idx]}/data.npy', f'{dir[idx]}/label.npy')

                data = np.vstack((data, d))
                mask = np.append(mask, m)
                labels = np.vstack((labels, l))
                i += 1

    return (data, mask), labels

In [34]:
train_data, train_labels, test_data, test_labels, val_data, val_labels = LoadDatasetRandom(CNN.Get_features_dir_frame(), CNN, LoadDatasetDirection_Frame, MAX_SEQ_LENGTH, 10)

CLASS: 0, 78_105
CLASS: 1, 68_91
CLASS: 2, 59_79
CLASS: 3, 267_357

CLASS: 0, 47_78
CLASS: 1, 40_68
CLASS: 2, 35_59
CLASS: 3, 160_267

CLASS: 0, 0_47
CLASS: 1, 0_40
CLASS: 2, 0_35
CLASS: 3, 0_160



Frames Grupales

In [12]:
def LoadDatasetDirection_Frames(dir, cnn, seq, stride, augmented):
    size = len(dir)
    data = np.zeros((0, seq, cnn.Get_feature_num()))
    mask = np.zeros((0, seq))
    labels = np.zeros((0, 1))
    
    for idx in range(size):
        print(f'{idx}/{size}')
        d, m, l = LoadVideo(f'{dir[idx]}/data.npy', f'{dir[idx]}/label.npy', seq, stride, cnn.Get_feature_num())
        data = np.vstack((data, d))
        mask = np.vstack((mask, m))
        labels = np.vstack((labels, l))        
        
        if (augmented):
            i = 0
            while (os.path.exists(f'{dir[idx]}/data_m{i}.npy') and (i <= 5 or l[0,0] != 3)):            
                d, m, l = LoadVideo(f'{dir[idx]}/data_m{i}.npy', f'{dir[idx]}/label.npy', seq, stride, cnn.Get_feature_num())

                data = np.vstack((data, d))
                mask = np.vstack((mask, m))
                labels = np.vstack((labels, l))
                i += 1
                
    return (data, mask), labels

In [29]:
train_data, train_labels, test_data, test_labels, val_data, val_labels = LoadDatasetRandom(CNN.Get_features_dir_ind(), CNN, LoadDatasetDirection_Frames, MAX_SEQ_LENGTH, 30)

CLASS: 0, 5_7
CLASS: 1, 11_15
CLASS: 2, 4_6
CLASS: 3, 54_72

CLASS: 0, 3_5
CLASS: 1, 6_11
CLASS: 2, 2_4
CLASS: 3, 32_54

CLASS: 0, 0_3
CLASS: 1, 0_6
CLASS: 2, 0_2
CLASS: 3, 0_32

0/43
1/43
2/43
3/43
4/43
5/43
6/43
7/43
8/43
9/43
10/43
11/43
12/43
13/43
14/43
15/43
16/43
17/43
18/43
19/43
20/43
21/43
22/43
23/43
24/43
25/43
26/43
27/43
28/43
29/43
30/43
31/43
32/43
33/43
34/43
35/43
36/43
37/43
38/43
39/43
40/43
41/43
42/43
0/31
1/31
2/31
3/31
4/31
5/31
6/31
7/31
8/31
9/31
10/31
11/31
12/31
13/31
14/31
15/31
16/31
17/31
18/31
19/31
20/31
21/31
22/31
23/31
24/31
25/31
26/31
27/31
28/31
29/31
30/31
0/26
1/26
2/26
3/26
4/26
5/26
6/26
7/26
8/26
9/26
10/26
11/26
12/26
13/26
14/26
15/26
16/26
17/26
18/26
19/26
20/26
21/26
22/26
23/26
24/26
25/26


Expand data

In [14]:
def ReduceTargets(tensor):
    column = tensor[:, 0]
    column[column < 3] = 0
    column[column >= 3] = 1
    return tensor

def CombineTargets(tensor):
    column = tensor[:, 0]
    column[column == 2] = 1
    column[column >= 3] = 2
    return tensor
def ArrayExpand(tensor):
    count = tensor.shape[0]
    for idx in range(count):
        print(f"i:{idx} to {count}")
        tensor = np.concatenate((tensor, [np.flip(tensor[idx])]), axis=0)
    return tensor

def Contar(tensor):
    unique_values, counts = np.unique(tensor, return_counts=True)
    totalSum = np.sum(counts)
    for value, count in zip(unique_values, counts):
        print(f"Clases {value}: {count} cantidad, {(count / totalSum) * 100:.2f}%")
    return counts

def RemoveData(data, labels, label, amount):
    i = 0
    a = 0
    size, c = labels.shape
    while (i < size and a < amount):
        if (labels[i,0] == label):

            #delete
            data = [np.delete(data[0], i, axis=0), np.delete(data[1], i, axis=0)]
            labels = np.delete(labels, i, axis=0)

            a = a + 1
            print(f'{a}/{amount}')
        else:
            i = i + 1
    
    return data, labels

def TransferData(data, labels, n_data, n_labels, label, amount):
    i = 0
    a = 0
    size, c = labels.shape
    while (i < size and a < amount):
        if (labels[i,0] == label):
            row = data[0][i]
            n_data[0] = np.append(n_data[0], [row], axis=0)

            row = data[1][i]
            n_data[1] = np.append(n_data[1], [row], axis=0)

            row = labels[i]
            n_labels = np.append(n_labels, [row], axis=0)

            #delete
            data = [np.delete(data[0], i, axis=0), np.delete(data[1], i, axis=0)]
            labels = np.delete(labels, i, axis=0)

            a = a + 1
            
        i = i + 1
    
    return data, labels, n_data, n_labels

In [14]:
#train_labels = ReduceTargets(train_labels)
#test_labels = ReduceTargets(test_labels)

#train_labels = CombineTargets(train_labels)
#test_labels = CombineTargets(test_labels)

#train_data = ArrayExpand(train_data)
#train_labels = ArrayExpand(train_labels)

#test_data = ArrayExpand(test_data)
#test_labels = ArrayExpand(test_labels)

Data information

Definition of validation data

In [30]:
print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")
print(f"Frame labels in train set: {train_labels.shape}")
Contar(train_labels)
print()
print(f"Frame features in test set: {test_data[0].shape}")
print(f"Frame masks in test set: {test_data[1].shape}")
print(f"Frame labels in test set: {test_labels.shape}")
Contar(test_labels)
print()
print(f"Frame features in validation set: {val_data[0].shape}")
print(f"Frame masks in validation set: {val_data[1].shape}")
print(f"Frame labels in validation set: {val_labels.shape}")
Contar(val_labels)
print()
#print(f"Frame features in holdout set: {holdout_data[0].shape}")
#print(f"Frame masks in holdout set: {holdout_data[1].shape}")
#print(f"Frame labels in holdout set: {holdout_labels.shape}")
#Contar(holdout_labels)
#print()

Frame features in train set: (617, 150, 960)
Frame masks in train set: (617, 150)
Frame labels in train set: (617, 1)
Clases 0.0: 44 cantidad, 7.13%
Clases 1.0: 84 cantidad, 13.61%
Clases 2.0: 12 cantidad, 1.94%
Clases 3.0: 477 cantidad, 77.31%

Frame features in test set: (368, 150, 960)
Frame masks in test set: (368, 150)
Frame labels in test set: (368, 1)
Clases 0.0: 25 cantidad, 6.79%
Clases 1.0: 58 cantidad, 15.76%
Clases 2.0: 36 cantidad, 9.78%
Clases 3.0: 249 cantidad, 67.66%

Frame features in validation set: (285, 150, 960)
Frame masks in validation set: (285, 150)
Frame labels in validation set: (285, 1)
Clases 0.0: 28 cantidad, 9.82%
Clases 1.0: 60 cantidad, 21.05%
Clases 2.0: 9 cantidad, 3.16%
Clases 3.0: 188 cantidad, 65.96%



In [16]:
np.save(f"Features/train_data.npy", train_data[0], fix_imports=True, allow_pickle=False)
np.save(f"Features/train_data_mask.npy", train_data[1], fix_imports=True, allow_pickle=False)
np.save(f"Features/train_labels.npy", train_labels, fix_imports=True, allow_pickle=False)

np.save(f"Features/test_data.npy", test_data[0], fix_imports=True, allow_pickle=False)
np.save(f"Features/test_data_mask.npy", test_data[1], fix_imports=True, allow_pickle=False)
np.save(f"Features/test_labels.npy", test_labels, fix_imports=True, allow_pickle=False)

np.save(f"Features/validation_data.npy", val_data[0], fix_imports=True, allow_pickle=False)
np.save(f"Features/validation_data_mask.npy", val_data[1], fix_imports=True, allow_pickle=False)
np.save(f"Features/validation_labels.npy", val_labels, fix_imports=True, allow_pickle=False)

Model Training

GRU

In [16]:
def get_sequence_model_GRU(seq_, num_feature_, layers):
    #num_feature_ = 4
    frame_features_input = keras.Input((seq_, num_feature_))
    mask_input = keras.Input((seq_,), dtype="bool")

    x = keras.layers.GRU(layers[0], return_sequences=True)(#64
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(layers[1])(x)#64
    x = keras.layers.Dropout(0.4)(x)

    
    layer_len = layers.shape[0]
    for i in range(2, layer_len):
        x = keras.layers.Dense(layers[i], activation="relu")(x)
        
    output = keras.layers.Dense(CLASSES, activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

def run_experiment_GRU(DIR, seq_, num_feature_, layers):
    #filepath = f"Index{INDEX}/_model/video_classifier"

    checkpoint = keras.callbacks.ModelCheckpoint(
        DIR, save_weights_only=True, save_best_only=True, verbose=1,
        mode='min', monitor='val_loss'
    )

    seq_model = get_sequence_model_GRU(seq_, num_feature_, layers)
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_data=([val_data[0], val_data[1]], val_labels),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[checkpoint],
    )
    seq_model.load_weights(DIR)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model

In [16]:
#layers = np.array((64,64,32,0))
layers = np.array((2,2,2,0))
HISTORY, MODEL = run_experiment_GRU(MODEL_DIR + "/video_classifier", MAX_SEQ_LENGTH, CNN.Get_feature_num(), layers)

KeyboardInterrupt: 

LSTM

In [17]:
def get_sequence_model_LSTM(seq_, num_feature_, layers):
    #num_feature_ = 4
    frame_features_input = keras.Input((seq_, num_feature_))
    mask_input = keras.Input((seq_,), dtype="bool")

    x = keras.layers.LSTM(layers[0], return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.LSTM(layers[1])(x)
    x = keras.layers.Dropout(0.4)(x)

    layer_len = layers.shape[0]
    for i in range(2, layer_len):
        x = keras.layers.Dense(layers[i], activation="relu")(x)
    
    output = keras.layers.Dense(CLASSES, activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

def run_experiment_LSTM(DIR, seq_, num_feature_, layers):
    checkpoint = keras.callbacks.ModelCheckpoint(
        DIR, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model_LSTM(seq_, num_feature_, layers)
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_data=([val_data[0], val_data[1]], val_labels),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(DIR)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model

In [16]:
layers = np.array((8,8,8,0))
HISTORY, MODEL = run_experiment_LSTM(f'video_classifier', MAX_SEQ_LENGTH, CNN.Get_feature_num(), layers)

Epoch 1/50
28/28 [==============================] - ETA: 0s - loss: 1.3774 - accuracy: 0.3300
Epoch 1: val_loss improved from inf to 1.33379, saving model to video_classifier
28/28 [==============================] - 14s 279ms/step - loss: 1.3774 - accuracy: 0.3300 - val_loss: 1.3338 - val_accuracy: 0.4516
Epoch 2/50
28/28 [==============================] - ETA: 0s - loss: 1.3160 - accuracy: 0.5000
Epoch 2: val_loss improved from 1.33379 to 1.27465, saving model to video_classifier
28/28 [==============================] - 5s 173ms/step - loss: 1.3160 - accuracy: 0.5000 - val_loss: 1.2747 - val_accuracy: 0.5806
Epoch 3/50
28/28 [==============================] - ETA: 0s - loss: 1.2764 - accuracy: 0.5418
Epoch 3: val_loss improved from 1.27465 to 1.22797, saving model to video_classifier
28/28 [==============================] - 5s 178ms/step - loss: 1.2764 - accuracy: 0.5418 - val_loss: 1.2280 - val_accuracy: 0.5484
Epoch 4/50
28/28 [==============================] - ETA: 0s - loss: 1.228

TRANSFORMER

In [18]:
class PositionalEmbedding(keras.layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = keras.layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask
    
class TransformerEncoder(keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [keras.layers.Dense(dense_dim, activation=tf.nn.gelu),
             keras.layers.Dense(embed_dim),]
        )
        self.layernorm_1 = keras.layers.LayerNormalization()
        self.layernorm_2 = keras.layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [19]:
def get_sequence_model_TRANSFORMER(seq_, num_feature_, layers):
    embed_dim = num_feature_
    dense_dim = int(layers[0])#4
    num_heads = int(layers[1])#1
    classes = len(np.unique(train_labels[:, 0]))

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        seq_, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = keras.layers.GlobalMaxPooling1D()(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model

def run_experiment_TRANSFORMER(DIR, seq_, num_feature_, layers):
    checkpoint = keras.callbacks.ModelCheckpoint(
        DIR, save_weights_only=True, save_best_only=True, verbose=1
    )
    seq_model = get_sequence_model_TRANSFORMER(seq_, num_feature_, layers)
    history = seq_model.fit(
        train_data[0],
        train_labels,
        validation_data=(val_data[0], val_labels),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(DIR)
    _, accuracy = seq_model.evaluate(test_data[0], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model

In [61]:
layers = np.array((4,1))
HISTORY, MODEL = run_experiment_TRANSFORMER(f'Test/video_classifier', MAX_SEQ_LENGTH, CNN.Get_feature_num(), layers)

!!!!!!!!!!960
Epoch 1/40
 28/207 [===>..........................] - ETA: 4:31 - loss: 1.6772 - accuracy: 0.6429

KeyboardInterrupt: 

Dense

In [20]:
def get_sequence_model_DENSE(num_feature_, layers):
    layer_size = np.size(layers)
    
    dense_model = keras.Sequential()

    dense_model.add(keras.layers.Dense(layers[0], activation='relu', name='dense_0', input_dim=num_feature_))

    for i in range(1, layer_size):
        if (layers[i] >= 64):
            dense_model.add(keras.layers.Dropout(0.3))
        dense_model.add(keras.layers.Dense(layers[i], name=f'dense_{i}',activation='relu'))

    dense_model.add(keras.layers.Dense(CLASSES, activation="softmax", name=f'dense_{layer_size}'))

    dense_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return dense_model

def run_experiment_DENSE(DIR, num_feature_, layers):
    checkpoint = keras.callbacks.ModelCheckpoint(
        DIR, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model_DENSE(num_feature_, layers)
    history = seq_model.fit(
        train_data[0],
        train_labels,
        validation_data=(val_data[0], val_labels),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(DIR)
    _, accuracy = seq_model.evaluate(test_data[0], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model

In [101]:
#layers = np.array((32,32,16,8))
layers = np.array((8,8))
HISTORY, MODEL = run_experiment_DENSE(f'video_classifier/', CNN.Get_feature_num(), layers)

Epoch 1/50
1375/1413 [============================>.] - ETA: 0s - loss: 0.2820 - accuracy: 0.9259
Epoch 1: val_loss improved from inf to 0.41182, saving model to video_classifier
1413/1413 [==============================] - 3s 2ms/step - loss: 0.2774 - accuracy: 0.9274 - val_loss: 0.4118 - val_accuracy: 0.9378
Epoch 2/50
1392/1413 [============================>.] - ETA: 0s - loss: 0.0638 - accuracy: 0.9861
Epoch 2: val_loss improved from 0.41182 to 0.32196, saving model to video_classifier
1413/1413 [==============================] - 1s 977us/step - loss: 0.0632 - accuracy: 0.9862 - val_loss: 0.3220 - val_accuracy: 0.9119
Epoch 3/50
1365/1413 [===========================>..] - ETA: 0s - loss: 0.0269 - accuracy: 0.9919
Epoch 3: val_loss improved from 0.32196 to 0.11689, saving model to video_classifier
1413/1413 [==============================] - 1s 967us/step - loss: 0.0271 - accuracy: 0.9920 - val_loss: 0.1169 - val_accuracy: 0.9482
Epoch 4/50
1355/1413 [===========================>..

KeyboardInterrupt: 

Complex

In [44]:
def get_model_COMPLEX():
    
    model = keras.Sequential(name='Seq_0')

    model.add(keras.layers.Dense(16, input_dim=CNN.Get_feature_num(), name='layer_0',activation='relu',))
    model.add(keras.layers.Dense(16, activation='relu', name='layer_1'))
    model.add(keras.layers.Dense(16, activation='relu', name='layer_2'))
    model.add(keras.layers.Dense(8, activation='relu', name='layer_3'))
    model.add(keras.layers.Dense(8, activation='relu', name='layer_4'))
    model.add(keras.layers.Dense(4, activation="softmax", name='layer_5'))
    
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    model.load_weights(f'../Models/Buenos/167_DENSE_MobileNetV3_Large_[16 16 16  8  8]_SEQ0_Feature_num960/video_classifier')
    return model
    
MODEL__ = get_model_COMPLEX()

In [79]:
def npy_recursive_reader(dir, data, labels):
    label_dir = f'{dir}\label.npy'

    if (os.path.exists(label_dir)):
        d = np.load(f'{dir}\data.npy')
        l = np.load(label_dir)
        l = np.ones((d.shape[0], 1)) * l[0]

        data = np.vstack((data, d))
        labels = np.vstack((labels, l))

    else:
        list = os.listdir(dir)

        for l in list:
            url = os.path.join(dir, l)
            if os.path.isdir(url):
                data, labels = npy_recursive_reader(url, data, labels)
                
    return data, labels

def LoadVideo__():
    data = np.zeros((0,CNN.Get_feature_num()))
    labels = np.zeros((0,1))

    return npy_recursive_reader(CNN.Get_features_dir_ind(), data, labels)

def Model_Report(model, data, labels):
    y_pred = model.predict(data, 0)
    y_pred_classes = np.argmax(y_pred, axis=1)

    _, test_accuracy = model.evaluate(data, labels, False)
    print('Test Accuracy: {:.2f}%'.format(test_accuracy * 100))

    report = classification_report(labels, y_pred_classes)
    print('Classification Report:\n', report)

    #ConfusionMatrix
    confusion_mtx = confusion_matrix(labels, y_pred_classes)
    print("Confusion matrix")
    print(confusion_mtx)
    
data, labels = LoadVideo__()
print(data.shape)
print(labels.shape)
#print(train_data[0].shape)
#print(train_labels.shape)
Model_Report(MODEL__, data, labels)

(26582, 960)
(26582, 1)
831/831 [==============================] - 2s 2ms/step - loss: 0.7494 - accuracy: 0.8039
Test Accuracy: 80.39%
Classification Report:
               precision    recall  f1-score   support

         0.0       0.53      0.91      0.67      1931
         1.0       0.53      0.59      0.56      4117
         2.0       0.41      0.63      0.50      1367
         3.0       0.98      0.85      0.91     19167

    accuracy                           0.80     26582
   macro avg       0.61      0.74      0.66     26582
weighted avg       0.85      0.80      0.82     26582

Confusion matrix
[[ 1748   183     0     0]
 [ 1276  2419   386    36]
 [    0   266   858   243]
 [  288  1700   834 16345]]


In [21]:
def LoadNewData(model, datas):
    n, s, f = datas[0].shape
    new_data = np.zeros((n, s, CLASSES))
    #single_example = test_data[0][14,:,:]  # Selecciona el primer ejemplo
    #single_example = single_example.reshape(1, 1024)
    for i in range(n):
        res = model.predict(datas[0][i,:,:], verbose=False)
        new_data[i,:,:] = res
    return new_data

train_data = (LoadNewData(MODEL__, train_data), train_data[1])
test_data = (LoadNewData(MODEL__,test_data), test_data[1])
val_data = (LoadNewData(MODEL__,val_data), val_data[1])

TRAINING

In [21]:
class Main_Model(ABC):

    @abstractmethod
    def Get_model_name():
        pass

    @abstractmethod
    def Run(dir, seq_, num_features_, layers_):
        pass
    @abstractmethod
    def Predict(model, data, verbose):
        pass
    @abstractmethod
    def Evaluate(model, data, labels, verbose):
        pass

class GRU(Main_Model):
    def Get_model_name():
        return 'GRU'    
    def Run(dir, seq_, num_features_, layers_):
        return run_experiment_GRU(dir, seq_, num_features_, layers_)
    def Predict(model, data, verbose):
        return model.predict(data, verbose)
    def Evaluate(model, data, labels, verbose):
        return model.evaluate(data, labels, verbose)
    
class LSTM(Main_Model):
    def Get_model_name():
        return 'LSTM'
    def Run(dir, seq_, num_features_, layers_):
        return run_experiment_LSTM(dir, seq_, num_features_, layers_)
    def Predict(model, data, verbose):
        return model.predict(data, verbose)
    def Evaluate(model, data, labels, verbose):
        return model.evaluate(data, labels, verbose)
    
class TRANSFORMER(Main_Model):
    def Get_model_name():
        return 'TRANSFORMER'
    def Run(dir, seq_, num_features_, layers_):
        return run_experiment_TRANSFORMER(dir, seq_, num_features_, layers_)
    def Predict(model, data, verbose):
        return model.predict(data[0], verbose)
    def Evaluate(model, data, labels, verbose):
        return model.evaluate(data[0], labels, verbose)
    
class DENSE(Main_Model):
    def Get_model_name():
        return 'DENSE'
    def Run(dir, seq_, num_features_, layers_):
        return run_experiment_DENSE(dir, num_features_, layers_)
    def Predict(model, data, verbose):
        return model.predict(data[0], verbose)
    def Evaluate(model, data, labels, verbose):
        return model.evaluate(data[0], labels, verbose)


def Testing(main, cnn):
    print(f'{main.Get_model_name()} {cnn.Get_model_name()} {cnn.Get_feature_num()}')

Testing(GRU, DenseNet121)
Testing(LSTM, DenseNet121)
Testing(DENSE, MobileNetV2)
Testing(TRANSFORMER, MobileNetV2)

GRU DenseNet121 1024
LSTM DenseNet121 1024
DENSE MobileNetV2 1280
TRANSFORMER MobileNetV2 1280


Report

In [22]:
DATA_FRAME = pd.DataFrame({'Index':[], 'Modelo_CNN':[],
                   'Modelo_Secuencia':[], 'Tam_Secuencia':[], 'Epocas':[],
                   'Dimension_capa1':[],'Dimension_capa2':[],
                   'Capa_densa1':[], 'Capa_densa2':[], 'Capa_salida':[],
                   'Precision':[],'Recall':[]})

def AddDataFrameInformation(index, cnn, model_seq, seq, epoc, layers, precision, recall):
    layer2 = 0
    layer3 = 0
    layer4 = 0
    if (len(layers) > 1):
        layer2 = layers[1]
    if (len(layers) > 2):
        layer3 = layers[2]
    if (len(layers) > 3):
        layer4 = layers[3]

    nueva_fila = {'Index':index, 'Modelo_CNN':cnn,
                  'Modelo_Secuencia':model_seq, 'Tam_Secuencia' : seq, 'Epocas':epoc,
                  'Dimension_capa1':layers[0], 'Dimension_capa2' : layer2,
                  'Capa_densa1':layer3, 'Capa_densa2' : layer4, 'Capa_salida': 4,
                  'Precision': precision, 'Recall' : recall,
                  }
    
    df = pd.read_csv('../Resumen.csv')
    df = df.append(nueva_fila, ignore_index=True)
    df.to_csv('../Resumen.csv', index=False)

display(DATA_FRAME)
#DATA_FRAME.to_csv('../Resumen.csv', index=False)

,Index,Modelo_CNN,Modelo_Secuencia,Tam_Secuencia,Epocas,Dimension_capa1,Dimension_capa2,Capa_densa1,Capa_densa2,Capa_salida,Precision,Recall


In [23]:
from sklearn.metrics import precision_score, recall_score

def InfoBase(num_features):    
    print("Hyperparameters:")
    print()
    print(f"MAX_SEQ_LENGTH: {MAX_SEQ_LENGTH}")
    print(f"NUM_FEATURES: {num_features}")
    print(f"IMG_SIZE: {IMG_SIZE}")
    print()
    print(f"BATCH_SIZE: {BATCH_SIZE}")
    print(f"TEST_RATIO: {TEST_RATIO}")
    print(f"VALIDATION_RATIO: {VALIDATION_RATIO}")
    print(f"HOLDOUT_RATIO: {HOLDOUT_RATIO}")
    print(f"CLASSES: {CLASSES}")
    print()
    print(f"EPOCHS: {EPOCHS}")
    print("_____________________________________")
    print("Data:")
    print()
    print(f"Frame features in train set: {train_data[0].shape}")
    print(f"Frame masks in train set: {train_data[1].shape}")
    print(f"Frame labels in train set: {train_labels.shape}")
    Contar(train_labels)
    print()
    print(f"Frame features in test set: {test_data[0].shape}")
    print(f"Frame masks in test set: {test_data[1].shape}")
    print(f"Frame labels in test set: {test_labels.shape}")
    Contar(test_labels)
    print()
    print(f"Frame features in validation set: {val_data[0].shape}")
    print(f"Frame masks in validation set: {val_data[1].shape}")
    print(f"Frame labels in validation set: {val_labels.shape}")
    Contar(val_labels)
    #print()
    #print(f"Frame features in holdout set: {holdout_data[0].shape}")
    #print(f"Frame masks in holdout set: {holdout_data[1].shape}")
    #print(f"Frame labels in holdout set: {holdout_labels.shape}")
    #Contar(holdout_labels)
    print("_____________________________________")

def GraphicsReport(history_, dir=""):
    train_accuracy = history_.history['accuracy']
    test_accuracy = history_.history['val_accuracy']
    validation_loss = history_.history['val_loss']

    epochs = range(1, len(train_accuracy) + 1)

    epoch_of_max_train_accuracy = history_.history['val_loss'].index(min(history_.history['val_loss'])) + 1
    plt.axvline(x=epoch_of_max_train_accuracy, color='red', linestyle='--', label=f"Best epoch: {epoch_of_max_train_accuracy}")

    plt.plot(epochs, train_accuracy, label=f"Train Accuracy")
    plt.plot(epochs, test_accuracy, label='Test Accuracy')
    plt.plot(epochs, validation_loss, label='Validation Loss')
    
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    if (len(dir) > 0):
        plt.savefig(f"{dir}/graph.jpg")
    else:
        plt.show()
    plt.clf()

    return epoch_of_max_train_accuracy

def GeneralReport(model, architecture, data, labels, f):
    sys.stdout = SYS_OUT
    y_pred = architecture.Predict(model, data, 0)
    y_pred_classes = np.argmax(y_pred, axis=1)

    _, test_accuracy = architecture.Evaluate(model, data, labels, False)
    sys.stdout = f
    print('Test Accuracy: {:.2f}%'.format(test_accuracy * 100))

    report = classification_report(labels, y_pred_classes)
    print('Classification Report:\n', report)
    
    pre = precision_score(labels, y_pred_classes, average=None)
    recall= recall_score(labels, y_pred_classes, average=None)

    #ConfusionMatrix
    confusion_mtx = confusion_matrix(labels, y_pred_classes)
    print("Confusion matrix")
    print(confusion_mtx)

    return np.round(pre, decimals=2), np.round(recall, decimals=2)

def ShowReport(history_, model_):
    #GeneralReport(model_, ffffff )
    model_.summary()
    GraphicsReport(history_)

def StrConvert(a):
    arrayString = np.array_str(a, precision=2)
    arrayString = "[" + arrayString[1:-1] + "]"

    return arrayString

def SaveReport(history, model, dir, index, architecture, cnn_, seq_, layers):

    with open(f"{dir}/report.txt", 'w') as f:
        sys.stdout = f
        print("Report with train")
        GeneralReport(model, architecture, train_data, train_labels, f)
        print("_____________________________")
        print("Report with test")
        pre, recall = GeneralReport(model, architecture, test_data, test_labels, f)
        print("_____________________________")
        print("Report with validation")
        GeneralReport(model, architecture, val_data, val_labels, f)
        #print("Report with holdout")
        #pre, recall = GeneralReport(model, architecture, holdout_data, holdout_labels)
        InfoBase(cnn_.Get_feature_num())
        model.summary()
        epoch = GraphicsReport(history, dir)
    sys.stdout = SYS_OUT

    pre = StrConvert(pre)
    recall = StrConvert(recall)
    AddDataFrameInformation(index, cnn_.Get_model_name(), architecture.Get_model_name(), seq_, epoch, layers, pre, recall)


In [24]:
def Model_Generator(architecture, cnn_, seq_, layers_):
    list_dir = os.listdir(MODEL_DIR)
    amount = sum(os.path.isdir(os.path.join(MODEL_DIR, x)) for x in list_dir)
    feature_num = cnn_.Get_feature_num()
    #feature_num = 4
    
    dir = MODEL_DIR + f"/{amount}_{architecture.Get_model_name()}_{cnn_.Get_model_name()}_{layers_}_SEQ{seq_}_Feature_num{feature_num}"
    if not os.path.exists(dir):
        os.makedirs(dir)
       
    history, model = architecture.Run(dir + "/video_classifier", seq_, feature_num, layers_)
    #history, model = model_.Run(dir + "/video_classifier", seq_, 4, layers_)
    SaveReport(history, model, dir, amount, architecture, cnn_, seq_, layers_)

Entrenamiento de modelo con frames grupales

In [43]:
Model_Generator(LSTM, CNN, MAX_SEQ_LENGTH , np.array((4,4,4)))
Model_Generator(GRU, CNN, MAX_SEQ_LENGTH, np.array((4,4,4)))

Model_Generator(LSTM, CNN, MAX_SEQ_LENGTH , np.array((4,4,4,4)))
Model_Generator(GRU, CNN, MAX_SEQ_LENGTH, np.array((4,4,4,4)))

Model_Generator(LSTM, CNN, MAX_SEQ_LENGTH , np.array((8,8,4)))
Model_Generator(GRU, CNN, MAX_SEQ_LENGTH, np.array((8,8,4)))

Model_Generator(LSTM, CNN, MAX_SEQ_LENGTH , np.array((8,8,8)))
Model_Generator(GRU, CNN, MAX_SEQ_LENGTH, np.array((8,8,8)))

Model_Generator(LSTM, CNN, MAX_SEQ_LENGTH , np.array((8,8,8,8)))
Model_Generator(GRU, CNN, MAX_SEQ_LENGTH, np.array((8,8,8,8)))

Model_Generator(LSTM, CNN, MAX_SEQ_LENGTH, np.array((16,16,8)))
Model_Generator(GRU, CNN, MAX_SEQ_LENGTH, np.array((16,16,8)))

Model_Generator(LSTM, CNN, MAX_SEQ_LENGTH, np.array((16,16,8,8)))
Model_Generator(GRU, CNN, MAX_SEQ_LENGTH, np.array((16,16,8,8)))

Model_Generator(LSTM, CNN, MAX_SEQ_LENGTH, np.array((32,32,16,0)))
Model_Generator(GRU, CNN, MAX_SEQ_LENGTH, np.array((32,32,16,0)))

Model_Generator(LSTM, CNN, MAX_SEQ_LENGTH, np.array((32,32,16,8)))
Model_Generator(GRU, CNN, MAX_SEQ_LENGTH, np.array((32,32,16,8)))

#Model_Generator(LSTM, CNN, MAX_SEQ_LENGTH, np.array((64,64,32,32,16)))
#Model_Generator(GRU, CNN, MAX_SEQ_LENGTH, np.array((64,64,32,32,16)))

#Model_Generator(LSTM, CNN, MAX_SEQ_LENGTH, np.array((128,64,32,16)))
#Model_Generator(GRU, CNN, MAX_SEQ_LENGTH, np.array((128,64,32,16)))

#Model_Generator(LSTM, CNN, MAX_SEQ_LENGTH, np.array((256,128,64,32)))
#Model_Generator(GRU, CNN, MAX_SEQ_LENGTH, np.array((256,128,64,32)))

Epoch 1/40
520/521 [============================>.] - ETA: 0s - loss: 0.8378 - accuracy: 0.7655
Epoch 1: val_loss improved from inf to 0.66514, saving model to ../Models/List/18_LSTM_MobileNetV3_Large_[4 4 4]_SEQ100_Feature_num960\video_classifier
521/521 [==============================] - 33s 49ms/step - loss: 0.8376 - accuracy: 0.7652 - val_loss: 0.6651 - val_accuracy: 0.7137
Epoch 2/40
520/521 [============================>.] - ETA: 0s - loss: 0.3107 - accuracy: 0.8954
Epoch 2: val_loss did not improve from 0.66514
521/521 [==============================] - 22s 42ms/step - loss: 0.3110 - accuracy: 0.8951 - val_loss: 0.9720 - val_accuracy: 0.7920
Epoch 3/40
521/521 [==============================] - ETA: 0s - loss: 0.2340 - accuracy: 0.9232
Epoch 3: val_loss did not improve from 0.66514
521/521 [==============================] - 22s 42ms/step - loss: 0.2340 - accuracy: 0.9232 - val_loss: 1.2581 - val_accuracy: 0.8181
Epoch 4/40
521/521 [==============================] - ETA: 0s - los

C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classifica

49/49 [==============================] - 1s 24ms/step - loss: 0.7421 - accuracy: 0.6624


C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classifica

41/41 [==============================] - 1s 22ms/step - loss: 0.6651 - accuracy: 0.7137


...

41/41 [==============================] - 1s 18ms/step - loss: 0.8111 - accuracy: 0.8181


C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classifica

Epoch 1/40
520/521 [============================>.] - ETA: 0s - loss: 0.4416 - accuracy: 0.8596
Epoch 1: val_loss improved from inf to 0.71898, saving model to ../Models/List/25_GRU_MobileNetV3_Large_[8 8 8]_SEQ100_Feature_num960\video_classifier
521/521 [==============================] - 27s 42ms/step - loss: 0.4410 - accuracy: 0.8598 - val_loss: 0.7190 - val_accuracy: 0.8235
Epoch 2/40
521/521 [==============================] - ETA: 0s - loss: 0.1059 - accuracy: 0.9634
Epoch 2: val_loss did not improve from 0.71898
521/521 [==============================] - 18s 35ms/step - loss: 0.1059 - accuracy: 0.9634 - val_loss: 1.1911 - val_accuracy: 0.8028
Epoch 3/40
521/521 [==============================] - ETA: 0s - loss: 0.0510 - accuracy: 0.9872
Epoch 3: val_loss did not improve from 0.71898
521/521 [==============================] - 18s 35ms/step - loss: 0.0510 - accuracy: 0.9872 - val_loss: 1.4831 - val_accuracy: 0.7897
Epoch 4/40
520/521 [============================>.] - ETA: 0s - loss

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [33]:
Model_Generator(TRANSFORMER, CNN, MAX_SEQ_LENGTH , np.array((4,1)))
Model_Generator(TRANSFORMER, CNN, MAX_SEQ_LENGTH, np.array((5,2)))

Model_Generator(TRANSFORMER, CNN, MAX_SEQ_LENGTH , np.array((8,2)))
Model_Generator(TRANSFORMER, CNN, MAX_SEQ_LENGTH, np.array((16,4)))

Model_Generator(TRANSFORMER, CNN, MAX_SEQ_LENGTH , np.array((32,8)))
Model_Generator(TRANSFORMER, CNN, MAX_SEQ_LENGTH, np.array((64,16)))

Model_Generator(TRANSFORMER, CNN, MAX_SEQ_LENGTH , np.array((64,32)))

Epoch 1/40
5/5 [==============================] - ETA: 0s - loss: 3.2990 - accuracy: 0.5235
Epoch 1: val_loss improved from inf to 3.47928, saving model to ../Models/List/193_TRANSFORMER_MobileNetV3_Large_[4 1]_SEQ150_Feature_num960\video_classifier
5/5 [==============================] - 15s 3s/step - loss: 3.2990 - accuracy: 0.5235 - val_loss: 3.4793 - val_accuracy: 0.6596
Epoch 2/40
5/5 [==============================] - ETA: 0s - loss: 2.2267 - accuracy: 0.7131
Epoch 2: val_loss improved from 3.47928 to 2.91140, saving model to ../Models/List/193_TRANSFORMER_MobileNetV3_Large_[4 1]_SEQ150_Feature_num960\video_classifier
5/5 [==============================] - 14s 3s/step - loss: 2.2267 - accuracy: 0.7131 - val_loss: 2.9114 - val_accuracy: 0.6596
Epoch 3/40
5/5 [==============================] - ETA: 0s - loss: 1.4068 - accuracy: 0.7536
Epoch 3: val_loss improved from 2.91140 to 1.33761, saving model to ../Models/List/193_TRANSFORMER_MobileNetV3_Large_[4 1]_SEQ150_Feature_num960\video

C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classifica

12/12 [==============================] - 2s 199ms/step - loss: 1.0734 - accuracy: 0.6766


C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classifica

9/9 [==============================] - 2s 205ms/step - loss: 0.8686 - accuracy: 0.6596


...

12/12 [==============================] - 4s 343ms/step - loss: 0.9449 - accuracy: 0.7120


C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classifica

9/9 [==============================] - 3s 350ms/step - loss: 0.7344 - accuracy: 0.7579


C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classifica

Epoch 1/40
5/5 [==============================] - ETA: 0s - loss: 4.5656 - accuracy: 0.5413
Epoch 1: val_loss improved from inf to 4.13517, saving model to ../Models/List/195_TRANSFORMER_MobileNetV3_Large_[8 2]_SEQ150_Feature_num960\video_classifier
5/5 [==============================] - 25s 5s/step - loss: 4.5656 - accuracy: 0.5413 - val_loss: 4.1352 - val_accuracy: 0.6596
Epoch 2/40
5/5 [==============================] - ETA: 0s - loss: 2.1019 - accuracy: 0.6645
Epoch 2: val_loss improved from 4.13517 to 1.57467, saving model to ../Models/List/195_TRANSFORMER_MobileNetV3_Large_[8 2]_SEQ150_Feature_num960\video_classifier
5/5 [==============================] - 24s 5s/step - loss: 2.1019 - accuracy: 0.6645 - val_loss: 1.5747 - val_accuracy: 0.6596
Epoch 3/40
5/5 [==============================] - ETA: 0s - loss: 1.0314 - accuracy: 0.7147
Epoch 3: val_loss improved from 1.57467 to 1.31205, saving model to ../Models/List/195_TRANSFORMER_MobileNetV3_Large_[8 2]_SEQ150_Feature_num960\video

Entrenamiento de modelo con frames individuales

In [51]:
Model_Generator(DENSE, CNN, 0, np.array([8]))
Model_Generator(DENSE, CNN, 0, np.array([16]))
Model_Generator(DENSE, CNN, 0, np.array([32]))
Model_Generator(DENSE, CNN, 0, np.array([64]))
Model_Generator(DENSE, CNN, 0, np.array([128]))
Model_Generator(DENSE, CNN, 0, np.array([256]))
Model_Generator(DENSE, CNN, 0, np.array([512]))
Model_Generator(DENSE, CNN, 0, np.array((1024,1024)))
Model_Generator(DENSE, CNN, 0, np.array((8,8)))
Model_Generator(DENSE, CNN, 0, np.array((16,16)))
Model_Generator(DENSE, CNN, 0, np.array((32,32)))
Model_Generator(DENSE, CNN, 0, np.array((64,64)))
Model_Generator(DENSE, CNN, 0, np.array((128,64)))
Model_Generator(DENSE, CNN, 0, np.array((256,128)))
Model_Generator(DENSE, CNN, 0, np.array((512,512)))
Model_Generator(DENSE, CNN, 0, np.array((1024,1024,1024)))
Model_Generator(DENSE, CNN, 0, np.array((8,8,8)))
Model_Generator(DENSE, CNN, 0, np.array((16,16,8)))
Model_Generator(DENSE, CNN, 0, np.array((32,32,16)))
Model_Generator(DENSE, CNN, 0, np.array((64,64,32)))
Model_Generator(DENSE, CNN, 0, np.array((128,64,32)))
Model_Generator(DENSE, CNN, 0, np.array((256,128,64)))
Model_Generator(DENSE, CNN, 0, np.array((512,512,512)))
Model_Generator(DENSE, CNN, 0, np.array((1024,1024,1024,1024)))
Model_Generator(DENSE, CNN, 0, np.array((8,8,8,8)))
Model_Generator(DENSE, CNN, 0, np.array((16,16,8,8)))
Model_Generator(DENSE, CNN, 0, np.array((32,32,16,16)))
Model_Generator(DENSE, CNN, 0, np.array((32,32,32,32)))
Model_Generator(DENSE, CNN, 0, np.array((64,64,32,32)))
Model_Generator(DENSE, CNN, 0, np.array((64,64,64,64)))
Model_Generator(DENSE, CNN, 0, np.array((128,64,32,16)))
Model_Generator(DENSE, CNN, 0, np.array((128,128,128,128)))
Model_Generator(DENSE, CNN, 0, np.array((256,128,64,32)))
Model_Generator(DENSE, CNN, 0, np.array((256,256,256,256)))
Model_Generator(DENSE, CNN, 0, np.array((512,512,256,128)))
Model_Generator(DENSE, CNN, 0, np.array((2048,1024,1024,512,256)))
Model_Generator(DENSE, CNN, 0, np.array((1024,1024,1024,1024,1024)))
Model_Generator(DENSE, CNN, 0, np.array((2048,2048,1024,1024,512)))
Model_Generator(DENSE, CNN, 0, np.array((8,8,8,8,8)))
Model_Generator(DENSE, CNN, 0, np.array((16,16,16,8,8)))
Model_Generator(DENSE, CNN, 0, np.array((32,32,16,16,8)))
Model_Generator(DENSE, CNN, 0, np.array((32,32,32,32,32)))
Model_Generator(DENSE, CNN, 0, np.array((64,64,32,32,16)))
Model_Generator(DENSE, CNN, 0, np.array((128,64,32,16,8)))
Model_Generator(DENSE, CNN, 0, np.array((128,128,128,128,128)))
Model_Generator(DENSE, CNN, 0, np.array((256,128,64,32,16)))
Model_Generator(DENSE, CNN, 0, np.array((1024,512,256,128,64)))
Model_Generator(DENSE, CNN, 0, np.array((2048,1024,512,256,128,64)))

Epoch 1/40
1/8 [==>...........................] - ETA: 1s - loss: 1.7675 - accuracy: 0.2520
Epoch 1: val_loss improved from inf to 1.12799, saving model to ../Models/List/629_DENSE_DenseNet121_[8]_SEQ0_Feature_num1024\video_classifier
8/8 [==============================] - 1s 101ms/step - loss: 1.5078 - accuracy: 0.3335 - val_loss: 1.1280 - val_accuracy: 0.5000
Epoch 2/40
1/8 [==>...........................] - ETA: 0s - loss: 1.3607 - accuracy: 0.3555
Epoch 2: val_loss improved from 1.12799 to 1.08216, saving model to ../Models/List/629_DENSE_DenseNet121_[8]_SEQ0_Feature_num1024\video_classifier
8/8 [==============================] - 0s 20ms/step - loss: 1.3046 - accuracy: 0.4071 - val_loss: 1.0822 - val_accuracy: 0.5000
Epoch 3/40
1/8 [==>...........................] - ETA: 0s - loss: 1.2392 - accuracy: 0.4355
Epoch 3: val_loss did not improve from 1.08216
8/8 [==============================] - 0s 4ms/step - loss: 1.1915 - accuracy: 0.4689 - val_loss: 1.0826 - val_accuracy: 0.5312
Epo

C:\Users\Francisco\AppData\Local\Temp\ipykernel_12668\2237941380.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


1/8 [==>...........................] - ETA: 1s - loss: 1.8665 - accuracy: 0.1875
Epoch 1: val_loss improved from inf to 1.23831, saving model to ../Models/List/630_DENSE_DenseNet121_[16]_SEQ0_Feature_num1024\video_classifier
8/8 [==============================] - 0s 29ms/step - loss: 1.4862 - accuracy: 0.2271 - val_loss: 1.2383 - val_accuracy: 0.4167
Epoch 2/40
1/8 [==>...........................] - ETA: 0s - loss: 1.3187 - accuracy: 0.3086
Epoch 2: val_loss improved from 1.23831 to 1.12212, saving model to ../Models/List/630_DENSE_DenseNet121_[16]_SEQ0_Feature_num1024\video_classifier
8/8 [==============================] - 0s 21ms/step - loss: 1.2699 - accuracy: 0.3332 - val_loss: 1.1221 - val_accuracy: 0.5208
Epoch 3/40
1/8 [==>...........................] - ETA: 0s - loss: 1.2201 - accuracy: 0.3730
Epoch 3: val_loss did not improve from 1.12212
8/8 [==============================] - 0s 4ms/step - loss: 1.1885 - accuracy: 0.4134 - val_loss: 1.1406 - val_accuracy: 0.5000
Epoch 4/40
1/

C:\Users\Francisco\AppData\Local\Temp\ipykernel_12668\2237941380.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


1/8 [==>...........................] - ETA: 2s - loss: 1.5956 - accuracy: 0.2656
Epoch 1: val_loss improved from inf to 1.05290, saving model to ../Models/List/631_DENSE_DenseNet121_[32]_SEQ0_Feature_num1024\video_classifier
8/8 [==============================] - 1s 44ms/step - loss: 1.3050 - accuracy: 0.3982 - val_loss: 1.0529 - val_accuracy: 0.5938
Epoch 2/40
1/8 [==>...........................] - ETA: 0s - loss: 1.0716 - accuracy: 0.5586
Epoch 2: val_loss improved from 1.05290 to 1.02062, saving model to ../Models/List/631_DENSE_DenseNet121_[32]_SEQ0_Feature_num1024\video_classifier
8/8 [==============================] - 0s 28ms/step - loss: 1.0203 - accuracy: 0.5872 - val_loss: 1.0206 - val_accuracy: 0.5417
Epoch 3/40
1/8 [==>...........................] - ETA: 0s - loss: 0.9347 - accuracy: 0.6152
Epoch 3: val_loss improved from 1.02062 to 0.97575, saving model to ../Models/List/631_DENSE_DenseNet121_[32]_SEQ0_Feature_num1024\video_classifier
8/8 [==============================] - 

...

7/8 [=========================>....] - ETA: 0s - loss: 1.2829 - accuracy: 0.3920
Epoch 1: val_loss improved from inf to 1.08313, saving model to ../Models/List/675_DENSE_DenseNet121_[1024  512  256  128   64]_SEQ0_Feature_num1024\video_classifier
8/8 [==============================] - 1s 107ms/step - loss: 1.2765 - accuracy: 0.3969 - val_loss: 1.0831 - val_accuracy: 0.5104
Epoch 2/40
7/8 [=========================>....] - ETA: 0s - loss: 0.9650 - accuracy: 0.5829
Epoch 2: val_loss improved from 1.08313 to 1.00994, saving model to ../Models/List/675_DENSE_DenseNet121_[1024  512  256  128   64]_SEQ0_Feature_num1024\video_classifier
8/8 [==============================] - 1s 80ms/step - loss: 0.9620 - accuracy: 0.5836 - val_loss: 1.0099 - val_accuracy: 0.5521
Epoch 3/40
7/8 [=========================>....] - ETA: 0s - loss: 0.7267 - accuracy: 0.7028
Epoch 3: val_loss did not improve from 1.00994
8/8 [==============================] - 0s 22ms/step - loss: 0.7242 - accuracy: 0.7028 - val_los

C:\Users\Francisco\AppData\Local\Temp\ipykernel_12668\2237941380.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


7/8 [=========================>....] - ETA: 0s - loss: 1.3008 - accuracy: 0.3677
Epoch 1: val_loss improved from inf to 1.05438, saving model to ../Models/List/676_DENSE_DenseNet121_[2048 1024  512  256  128   64]_SEQ0_Feature_num1024\video_classifier
8/8 [==============================] - 2s 171ms/step - loss: 1.2932 - accuracy: 0.3714 - val_loss: 1.0544 - val_accuracy: 0.5312
Epoch 2/40
7/8 [=========================>....] - ETA: 0s - loss: 1.0766 - accuracy: 0.5170
Epoch 2: val_loss did not improve from 1.05438
8/8 [==============================] - 0s 59ms/step - loss: 1.0731 - accuracy: 0.5189 - val_loss: 1.0655 - val_accuracy: 0.5833
Epoch 3/40
7/8 [=========================>....] - ETA: 0s - loss: 0.8517 - accuracy: 0.6239
Epoch 3: val_loss did not improve from 1.05438
8/8 [==============================] - 0s 57ms/step - loss: 0.8470 - accuracy: 0.6256 - val_loss: 1.2331 - val_accuracy: 0.5625
Epoch 4/40
7/8 [=========================>....] - ETA: 0s - loss: 0.6533 - accuracy: 

C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Francisco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classifica

<Figure size 640x480 with 0 Axes>

In [32]:
def ModelsGenerator():
    Model_Generator(DENSE, CNN, 0, np.array([8]))
    Model_Generator(DENSE, CNN, 0, np.array([16]))
    Model_Generator(DENSE, CNN, 0, np.array([32]))
    Model_Generator(DENSE, CNN, 0, np.array([64]))
    Model_Generator(DENSE, CNN, 0, np.array([128]))
    Model_Generator(DENSE, CNN, 0, np.array([256]))
    Model_Generator(DENSE, CNN, 0, np.array([512]))
    Model_Generator(DENSE, CNN, 0, np.array((1024,1024)))
    Model_Generator(DENSE, CNN, 0, np.array((8,8)))
    Model_Generator(DENSE, CNN, 0, np.array((16,16)))
    Model_Generator(DENSE, CNN, 0, np.array((32,32)))
    Model_Generator(DENSE, CNN, 0, np.array((64,64)))
    Model_Generator(DENSE, CNN, 0, np.array((128,64)))
    Model_Generator(DENSE, CNN, 0, np.array((256,128)))
    Model_Generator(DENSE, CNN, 0, np.array((512,512)))
    Model_Generator(DENSE, CNN, 0, np.array((1024,1024,1024)))
    Model_Generator(DENSE, CNN, 0, np.array((8,8,8)))
    Model_Generator(DENSE, CNN, 0, np.array((16,16,8)))
    Model_Generator(DENSE, CNN, 0, np.array((32,32,16)))
    Model_Generator(DENSE, CNN, 0, np.array((64,64,32)))
    Model_Generator(DENSE, CNN, 0, np.array((128,64,32)))
    #Model_Generator(DENSE, CNN, 0, np.array((256,128,64)))
    #Model_Generator(DENSE, CNN, 0, np.array((512,512,512)))
    #Model_Generator(DENSE, CNN, 0, np.array((1024,1024,1024,1024)))
    Model_Generator(DENSE, CNN, 0, np.array((8,8,8,8)))
    Model_Generator(DENSE, CNN, 0, np.array((16,16,8,8)))
    Model_Generator(DENSE, CNN, 0, np.array((32,32,16,16)))
    Model_Generator(DENSE, CNN, 0, np.array((32,32,32,32)))
    Model_Generator(DENSE, CNN, 0, np.array((64,64,32,32)))
    Model_Generator(DENSE, CNN, 0, np.array((64,64,64,64)))
    Model_Generator(DENSE, CNN, 0, np.array((128,64,32,16)))
    Model_Generator(DENSE, CNN, 0, np.array((128,128,128,128)))
    Model_Generator(DENSE, CNN, 0, np.array((256,128,64,32)))
    Model_Generator(DENSE, CNN, 0, np.array((256,256,256,256)))
    #Model_Generator(DENSE, CNN, 0, np.array((512,512,256,128)))
    #Model_Generator(DENSE, CNN, 0, np.array((2048,1024,1024,512,256)))
    #Model_Generator(DENSE, CNN, 0, np.array((1024,1024,1024,1024,1024)))
    #Model_Generator(DENSE, CNN, 0, np.array((2048,2048,1024,1024,512)))
    Model_Generator(DENSE, CNN, 0, np.array((8,8,8,8,8)))
    Model_Generator(DENSE, CNN, 0, np.array((16,16,16,8,8)))
    Model_Generator(DENSE, CNN, 0, np.array((32,32,16,16,8)))
    Model_Generator(DENSE, CNN, 0, np.array((32,32,32,32,32)))
    Model_Generator(DENSE, CNN, 0, np.array((64,64,32,32,16)))
    Model_Generator(DENSE, CNN, 0, np.array((128,64,32,16,8)))
    #Model_Generator(DENSE, CNN, 0, np.array((128,128,128,128,128)))
    #Model_Generator(DENSE, CNN, 0, np.array((256,128,64,32,16)))
    #Model_Generator(DENSE, CNN, 0, np.array((1024,512,256,128,64)))
    #Model_Generator(DENSE, CNN, 0, np.array((2048,1024,512,256,128,64)))

def HyperParameters1():
    global TRAIN_RATIO
    global TEST_RATIO
    global VALIDATION_RATIO
    global BATCH_SIZE

    TEST_RATIO = 0.3
    VALIDATION_RATIO = 0.2
    TRAIN_RATIO = 1 - TEST_RATIO - VALIDATION_RATIO
    BATCH_SIZE = 256

def HyperParameters2():
    global TRAIN_RATIO
    global TEST_RATIO
    global VALIDATION_RATIO
    global BATCH_SIZE

    TEST_RATIO = 0.25#0.15
    VALIDATION_RATIO = 0.15#0.1
    TRAIN_RATIO = 1 - TEST_RATIO - VALIDATION_RATIO
    BATCH_SIZE = 256

def HyperParameters3():
    global TRAIN_RATIO
    global TEST_RATIO
    global VALIDATION_RATIO
    global BATCH_SIZE
    
    TEST_RATIO = 0.15
    VALIDATION_RATIO = 0.1
    TRAIN_RATIO = 1 - TEST_RATIO - VALIDATION_RATIO
    BATCH_SIZE = 512
    
def HyperParameters4():
    global TRAIN_RATIO
    global TEST_RATIO
    global VALIDATION_RATIO
    global BATCH_SIZE
    
    TEST_RATIO = 0.1
    VALIDATION_RATIO = 0.05
    TRAIN_RATIO = 1 - TEST_RATIO - VALIDATION_RATIO
    BATCH_SIZE = 1024
    
def AutoTrain(cnn):
    global CNN
    global train_data
    global train_labels
    global test_data
    global test_labels
    global val_data
    global val_labels

    CNN = cnn
    HyperParameters1()
    train_data, train_labels, test_data, test_labels, val_data, val_labels = LoadDatasetRandom(CNN.Get_features_dir_frame(), CNN, LoadDatasetDirection_Frame, MAX_SEQ_LENGTH, 50)
    ModelsGenerator()

    HyperParameters2()
    train_data, train_labels, test_data, test_labels, val_data, val_labels = LoadDatasetRandom(CNN.Get_features_dir_frame(), CNN, LoadDatasetDirection_Frame, MAX_SEQ_LENGTH, 50)
    ModelsGenerator()

    HyperParameters3()
    train_data, train_labels, test_data, test_labels, val_data, val_labels = LoadDatasetRandom(CNN.Get_features_dir_frame(), CNN, LoadDatasetDirection_Frame, MAX_SEQ_LENGTH, 50)
    ModelsGenerator()
    
    HyperParameters4()
    train_data, train_labels, test_data, test_labels, val_data, val_labels = LoadDatasetRandom(CNN.Get_features_dir_frame(), CNN, LoadDatasetDirection_Frame, MAX_SEQ_LENGTH, 50)
    ModelsGenerator()

In [33]:
#AutoTrain(DenseNet121)
#AutoTrain(InceptionV3)
#AutoTrain(MobileNetV2)
#AutoTrain(ResNet101V2)
#AutoTrain(MobileNetV3)
AutoTrain(MobileNetV3_Large)

CLASS: 0, 84_105
CLASS: 1, 72_91
CLASS: 2, 63_79
CLASS: 3, 285_357

CLASS: 0, 52_84
CLASS: 1, 45_72
CLASS: 2, 39_63
CLASS: 3, 178_285

CLASS: 0, 0_52
CLASS: 1, 0_45
CLASS: 2, 0_39
CLASS: 3, 0_178

Epoch 1/40
 1/11 [=>............................] - ETA: 3s - loss: 1.5395 - accuracy: 0.2695
Epoch 1: val_loss improved from inf to 1.01437, saving model to ../Models/List/26_DENSE_MobileNetV3_Large_[8]_SEQ0_Feature_num960\video_classifier
11/11 [==============================] - 1s 28ms/step - loss: 1.2929 - accuracy: 0.4446 - val_loss: 1.0144 - val_accuracy: 0.6094
Epoch 2/40
 1/11 [=>............................] - ETA: 0s - loss: 1.1046 - accuracy: 0.5352
Epoch 2: val_loss improved from 1.01437 to 0.89996, saving model to ../Models/List/26_DENSE_MobileNetV3_Large_[8]_SEQ0_Feature_num960\video_classifier
11/11 [==============================] - 0s 13ms/step - loss: 1.0361 - accuracy: 0.6032 - val_loss: 0.9000 - val_accuracy: 0.7031
Epoch 3/40
 1/11 [=>............................] - ETA: 

C:\Users\Francisco\AppData\Local\Temp\ipykernel_12620\2237941380.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


 1/11 [=>............................] - ETA: 2s - loss: 1.6433 - accuracy: 0.2539
Epoch 1: val_loss improved from inf to 0.97982, saving model to ../Models/List/27_DENSE_MobileNetV3_Large_[16]_SEQ0_Feature_num960\video_classifier
11/11 [==============================] - 1s 30ms/step - loss: 1.2890 - accuracy: 0.4325 - val_loss: 0.9798 - val_accuracy: 0.5781
Epoch 2/40
 1/11 [=>............................] - ETA: 0s - loss: 1.0213 - accuracy: 0.5273
Epoch 2: val_loss improved from 0.97982 to 0.87950, saving model to ../Models/List/27_DENSE_MobileNetV3_Large_[16]_SEQ0_Feature_num960\video_classifier
11/11 [==============================] - 0s 16ms/step - loss: 0.9589 - accuracy: 0.6090 - val_loss: 0.8795 - val_accuracy: 0.6719
Epoch 3/40
 1/11 [=>............................] - ETA: 0s - loss: 0.7698 - accuracy: 0.7539
Epoch 3: val_loss improved from 0.87950 to 0.78428, saving model to ../Models/List/27_DENSE_MobileNetV3_Large_[16]_SEQ0_Feature_num960\video_classifier
11/11 [==========

C:\Users\Francisco\AppData\Local\Temp\ipykernel_12620\2237941380.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


 1/11 [=>............................] - ETA: 2s - loss: 2.2100 - accuracy: 0.1680
Epoch 1: val_loss improved from inf to 0.99040, saving model to ../Models/List/28_DENSE_MobileNetV3_Large_[32]_SEQ0_Feature_num960\video_classifier
11/11 [==============================] - 1s 29ms/step - loss: 1.4064 - accuracy: 0.3662 - val_loss: 0.9904 - val_accuracy: 0.6016
Epoch 2/40
 1/11 [=>............................] - ETA: 0s - loss: 1.0806 - accuracy: 0.4961
Epoch 2: val_loss improved from 0.99040 to 0.81769, saving model to ../Models/List/28_DENSE_MobileNetV3_Large_[32]_SEQ0_Feature_num960\video_classifier
11/11 [==============================] - 0s 15ms/step - loss: 0.9353 - accuracy: 0.6112 - val_loss: 0.8177 - val_accuracy: 0.7266
Epoch 3/40
 1/11 [=>............................] - ETA: 0s - loss: 0.7509 - accuracy: 0.7383
Epoch 3: val_loss improved from 0.81769 to 0.70554, saving model to ../Models/List/28_DENSE_MobileNetV3_Large_[32]_SEQ0_Feature_num960\video_classifier
11/11 [==========

...

1/5 [=====>........................] - ETA: 1s - loss: 1.4511 - accuracy: 0.2168
Epoch 1: val_loss improved from inf to 1.33061, saving model to ../Models/List/172_DENSE_MobileNetV3_Large_[64 64 32 32 16]_SEQ0_Feature_num960\video_classifier
5/5 [==============================] - 1s 69ms/step - loss: 1.3902 - accuracy: 0.2615 - val_loss: 1.3306 - val_accuracy: 0.6061
Epoch 2/40
1/5 [=====>........................] - ETA: 0s - loss: 1.3288 - accuracy: 0.4434
Epoch 2: val_loss improved from 1.33061 to 1.14927, saving model to ../Models/List/172_DENSE_MobileNetV3_Large_[64 64 32 32 16]_SEQ0_Feature_num960\video_classifier
5/5 [==============================] - 0s 53ms/step - loss: 1.2980 - accuracy: 0.4857 - val_loss: 1.1493 - val_accuracy: 0.6667
Epoch 3/40
1/5 [=====>........................] - ETA: 0s - loss: 1.2552 - accuracy: 0.4746
Epoch 3: val_loss improved from 1.14927 to 0.94322, saving model to ../Models/List/172_DENSE_MobileNetV3_Large_[64 64 32 32 16]_SEQ0_Feature_num960\video

C:\Users\Francisco\AppData\Local\Temp\ipykernel_12620\2237941380.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


1/5 [=====>........................] - ETA: 1s - loss: 1.5795 - accuracy: 0.1826
Epoch 1: val_loss improved from inf to 1.18405, saving model to ../Models/List/173_DENSE_MobileNetV3_Large_[128  64  32  16   8]_SEQ0_Feature_num960\video_classifier
5/5 [==============================] - 1s 91ms/step - loss: 1.4000 - accuracy: 0.3132 - val_loss: 1.1840 - val_accuracy: 0.5455
Epoch 2/40
1/5 [=====>........................] - ETA: 0s - loss: 1.2589 - accuracy: 0.4746
Epoch 2: val_loss improved from 1.18405 to 0.96093, saving model to ../Models/List/173_DENSE_MobileNetV3_Large_[128  64  32  16   8]_SEQ0_Feature_num960\video_classifier
5/5 [==============================] - 0s 61ms/step - loss: 1.2264 - accuracy: 0.4545 - val_loss: 0.9609 - val_accuracy: 0.5455
Epoch 3/40
1/5 [=====>........................] - ETA: 0s - loss: 1.1521 - accuracy: 0.4609
Epoch 3: val_loss improved from 0.96093 to 0.89585, saving model to ../Models/List/173_DENSE_MobileNetV3_Large_[128  64  32  16   8]_SEQ0_Featu

C:\Users\Francisco\AppData\Local\Temp\ipykernel_12620\2237941380.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(nueva_fila, ignore_index=True)


<Figure size 640x480 with 0 Axes>

In [ ]:
import re

def EsMayor(string):
    #s = string.split('[')[-1].split(']')[0].split(' ')
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", string)
    numbers = [float(num) for num in numbers]
    for i in range(len(numbers)):
        if (numbers[i] < 0.6):
            return None
    return numbers

    
pd_result = pd.read_csv(f'../Resumen.csv')

Index = pd_result["Index"].values.tolist()
Precision = pd_result["Precision"].values.tolist()
Recall = pd_result["Recall"].values.tolist()

size = len(Index)
for i in range(size):
    p = EsMayor(Precision[i])
    r = EsMayor(Recall[i])
    if (p != None and r != None):
        print(f'{i:2}: {Index[i]:3} | {p}: {min(p):.2f} | {r}: {min(r):.2f} ')

126: 152 | [0.75, 0.67, 0.78, 0.89]: 0.67 | [0.6, 0.67, 0.88, 0.92]: 0.60 
